<div style='float: right'><img src='pic/eisbahn.png'/></div>
## <div id='eisbahn' />アイスバーン

In [3]:
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt
from itertools import product
from pulp import *
from unionfind import unionfind
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
 . . . . . 
R . .*.*L .
 . . . . . 
. . . . . .
 . . U . . 
.*. .*. . .
 . . . . . 
. . . . . .
 . . . . . 
. R*.*. . R
 . . . . . """.split('\n')
nw, nh = len(data[0])//2, len(data)//2

### 問題
* INから入りOUTにいく１本の線をひきます
* 灰色のマスをアイスバーンとし、必ず通ります
* アイスバーンで曲がってはいけません
* アイスバーンのみ交差可です
* 矢印は必ず通ること

### 変数
* vh：0:L, 1:R (1)
* vv：0:U, 1:D (2)
* vhs (3)
* vvs (4)

### 制約
* vhsをvhで、vvsをvvで表現 (5)
* 矢印は必ず通ること (6)
* 各マスで入る数と出る数が同じこと (7)
* アイスバーンなら横は同じ、縦も同じこと（曲がらない） (8)
* アイスバーンなら線は2以上 (9)
* アイスバーンでないなら線は2以下 (10)
* 線は1つ (11)

In [8]:
m = LpProblem()
vh = addbinvars(nh, nw+1, 2) # 0:L, 1:R (1)
vv = addbinvars(nh+1, nw, 2) # 0:U, 1:D (2)
vhs = addvars(nh, nw+1) # (3)
vvs = addvars(nh+1, nw) # (4)
for i, j in product(range(nh), range(nw+1)):
    c = data[i*2+1][j*2]
    m += lpDot([1,2], vh[i][j]) == vhs[i][j] # (5)
    if c != '.' or j%nw == 0:
        m += vhs[i][j] == (1 if c=='L' else 2 if c=='R' else 0) # (6)
for i, j in product(range(nh+1), range(nw)):
    c = data[i*2][j*2+1]
    m += lpDot([1,2], vv[i][j]) == vvs[i][j] # (5)
    if c != '.' or i%nh == 0:
        m += vvs[i][j] == (1 if c=='U' else 2 if c=='D' else 0) # (6)
for i, j in product(range(nh), range(nw)):
    e1 = lpSum(vv[i+k][j][1-k] + vh[i][j+k][1-k] for k in range(2))
    e2 = lpSum(vv[i+k][j][k]   + vh[i][j+k][k] for k in range(2))
    m += e1 == e2 # (7)
    if data[i*2+1][i*2+1] == '*':
        m += vhs[i][j] == vhs[i][j+1] # (8)
        m += vvs[i][j] == vvs[i+1][j] # (8)
        m += e1 + e2 >= 2 # (9)
    else:
        m += e1 + e2 <= 2 # (10)
while True:
    %time m.solve()
    rhs = np.vectorize(value)(vhs)
    rvs = np.vectorize(value)(vvs)
    if m.status != 1: break
    break
    b = [[all([value(vhs[i + k][j]) > 0.5 and value(vvs[i][j + k]) > 0.5 for k in r2]) for j in rh] for i in rw]
    e = [[all([value(vhs[i + k][j]) < 0.5 and value(vvs[i][j + k]) < 0.5 for k in r2]) for j in rh] for i in rw]
    u = unionfind(nh * nw)
    p = -1
    for i in rw:
        for j in rh:
            if not e[i][j]: p = i + j * nw
            if b[i][j]:
                u.unite(i - 1 + j * nw, i + 1 + j * nw)
                u.unite(i + j * nw - nw, i + j * nw + nw)
            else:
                if i > 0 and value(vhs[i][j]) > 0.5 and not b[i - 1][j]:
                    u.unite(i + j * nw, i - 1 + j * nw)
                if j > 0 and value(vvs[i][j]) > 0.5 and not b[i][j - 1]:
                    u.unite(i + j * nw, i + j * nw - nw)
    if all([b[i][j] or e[i][j] or u.issame(p, i + j * nw) for i in rw for j in rh]): break
    for gr in u.groups():
        if len(gr) == 1: continue
        s = []
        for g in gr:
            i, j = g % nw, g // nw
            for k in r2:
                for l in r2:
                    if value(vh[i + k][j][l]) > 0.5: s.append(vh[i + k][j][l])
                    if value(vv[i][j + k][l]) > 0.5: s.append(vv[i][j + k][l])
        m += lpSum(s) <= len(s) - 2 # (11)
for j in rh1:
    for i in rw:
        sys.stdout.write(' %c' % '.UD'[int(value(vvs[i][j]))])
    sys.stdout.write('\n')
    if j == nh: break
    for i in rw1:
        sys.stdout.write('%c%c' % ('.LR'[int(value(vhs[i][j]))],
            '\n' if i == nw else ch[j * 2 + 1][i * 2 + 1]))

Wall time: 27 ms


In [10]:
(rhs>0)*2

array([[2, 0, 0, 0, 2, 0],
       [0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 2, 0],
       [0, 2, 2, 2, 2, 2]])